# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-10-20 21:23:25] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-10-20 21:23:25] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-10-20 21:23:25] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-10-20 21:23:25] INFO trace.py:48: opentelemetry package is not installed, tracing disabled


`torch_dtype` is deprecated! Use `dtype` instead!


...

Capturing batches (bs=120 avail_mem=53.62 GB):   5%|▌         | 1/20 [00:00<00:03,  6.02it/s]

Capturing batches (bs=80 avail_mem=53.60 GB):  20%|██        | 4/20 [00:00<00:01, 14.71it/s]

Capturing batches (bs=40 avail_mem=53.57 GB):  50%|█████     | 10/20 [00:00<00:00, 20.06it/s]

Capturing batches (bs=8 avail_mem=53.55 GB):  80%|████████  | 16/20 [00:00<00:00, 20.67it/s] 

Capturing batches (bs=1 avail_mem=53.53 GB): 100%|██████████| 20/20 [00:00<00:00, 20.68it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Kelsey and I am a 19 year old college student. I am now a medical student and I am preparing to attend medical school. 

I have heard about the Discovery Innovation program which allows students to get funding for their research. I heard that the Discovery Innovation Program helps students who are developing a medical device and want to enter into the market. 

I am a bit confused about whether or not the Discovery Innovation program is my best option. The program is offered by the University of Maryland. However, I am interested in a different field of study, and I believe that there are many students who are also interested in the medical field
Prompt: The president of the United States is
Generated text:  5 feet 10 inches tall. The vice president is 3 feet 7 inches tall. If the president is 1 inch taller than the vice president, how tall is the vice president? To determine the height of the vice president, we need to follow these steps:

1.

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [Age] year old [Occupation]. I'm currently [Current Location] and I enjoy [Favorite Activity/Interest]. I'm a [Favorite Hobby/Interest] and I love [Favorite Food/Drink]. I'm [Favorite Color] and I have a [Favorite Animal/Plant]. I'm [Favorite Book/Article/Video] and I'm always [Favorite Quote/Advice/Tip]. I'm [Favorite Movie/TV Show/Book/Artist]. I'm [Favorite Sport/Activity/Job]. I'm [Favorite Place/Event/Person]. I'm [Favorite Thing to Do

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. 

A. True
B. False
A. True

Paris is the capital city of France, and it is known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. The city is also famous for its rich history, including the French Revolution and the French Revolution Monument. Paris is a major cultural and economic center in Europe and is home to many famous museums, theaters, and restaurants. The city is also known for its fashion industry, with many famous fashion designers and boutiques located in the city. Overall, Paris is a vibrant and dynamic city that is a must

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. Some potential future trends include:

1. Increased integration with other technologies: AI is likely to become more integrated with other technologies such as IoT, blockchain, and quantum computing, creating new possibilities for new applications and services.

2. Enhanced privacy and security: As AI becomes more integrated with other technologies, there will be increased concerns about privacy and security. There will be a need for new approaches to data protection and privacy, as well as new technologies to ensure that AI systems are secure and reliable.

3. Greater emphasis on ethical considerations:



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name], and I'm a friendly and knowledgeable traveler looking to share my knowledge about the world. I'm always eager to learn new things and explore new cultures, and I love sharing my experiences with others. I'm a bit of a nomad by nature, and I love traveling and experiencing new places. Whether I'm wandering around a bustling city or hiking through dense forests, I always try to make the most of my time and find the best spots to experience the local culture and cuisine. I'm also a bit of a language learner, and I love to practice my English or other foreign languages whenever I can. Overall, I'm

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, the city known for its historical landmarks, architecture, and vibrant cultural scene. It is the capital of the country and is home to the Eiffel Tower, Not

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

].

 I

...

 blockchain

.

 This

 integration

 will

In [6]:
llm.shutdown()